<a href="https://colab.research.google.com/github/valery-kustov/FinancialAnalysis/blob/main/Ratings_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import re
import warnings
import clipboard
import numpy as np
import pandas as pd
import pickle
import pathlib
import datetime

from sklearn.model_selection import GroupShuffleSplit, StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import classification_report

### Prepare ratings

In [ ]:
tickers = {
    'Лукойл': "LKOH", 'СИБУР Холдинг': None, 'ЧТПЗ': "CHEP", 'ФосАгро': "PHOR", 'Газпром': "GAZP",
    'Группа ЛСР (ПАО)': "LSRG", 'ПКТ': None, 'НОВАТЭК': "NVTK", 'Металлоинвест': None, 'АФК Система': "AFKS",
    'ГТЛК': None, 'РУСАЛ': "RUAL", 'Полюс': "PLZL", 'АЛРОСА': "ALRS", 'Кокс': "KSGR", 'МТС': "MTSS", 
    'РЖД': None, 'Роснано': None, 'ФГУП Почта России': None, 'ФПК': None, 'Ростелеком': "RTKM", 'Вымпелком': "VEON",
    'ТГК-1': "TGKA", 'Северсталь': "CHMF", 'ЛЕНТА': "LNTA", 'Атомэнергопром': None, 'Башнефть': "BANE", 
    'ГМК Норильский никель': "GMKN", 'Петропавловск': "POGR", 'ТрансКонтейнер': "TRCN", 'Теле2-Санкт-Петербург': None,
    'Совкомфлот': "FLOT", 'Акрон': "AKRN", 'ДВМП (FESCO)': "FESH", 'МБЭС': None, 'ЛК Европлан': None,
    'ЕвроХим': None, 'НЛМК': "NLMK", 'Нордголд': "RTSD", 'Буровая компания Евразия': None, 'ФСК ЕЭС': "FEES",
    'РусГидро': "HYDR", 'Белуга Групп': "BELU", 'Газпром нефть': "SIBN", 'ДелоПортс': None, 'ГИДРОМАШСЕРВИС': None,
    'Борец Интернэшнл': None, 'Россети Московский регион': "MSRS", 'СУЭК': None, 'Почта России': None,
    'МегаФон': "MFON", 'Первая Грузовая Компания ПАО': None, 'Энел Россия': "ENRU", 'Новая перевозочная компания': None, 
    'Автодор': None, 'Евраз': "EVRE", 'Каркаде': None, 'X5 Retail Group': "FIVE", 'ММК': "MAGN", 'ОГК-2': "OGKB",
    'Уралкалий': "URKA",  'ОКЕЙ': "OKEY"
}

fin_tickers = [
    'РОСБАНК', 'Газпромбанк', 'Банк Интеза', 'Кредит Европа Банк', 'ЛОКО-Банк', 
    'Банк ДОМ.РФ', 'Совкомбанк', 'Россельхозбанк', 'Сбербанк России', 'Альфа-Банк',
    'ДОМ.РФ', 'Промсвязьбанк', 'ХКФ Банк', 'Тинькофф Банк', 'Тойота Банк', 'Банк ЗЕНИТ',
    'АБСОЛЮТ БАНК', 'Банк ФК Открытие', 'Совкомбанк Лизинг', 'Банк Санкт-Петербург', 
    'Фольксваген Банк РУС', 'Ипотечный агент АИЖК 2013-1', 'Ипотечный агент МКБ', 'ДельтаКредит',     
    'ВЭБ-лизинг', 'ВЭБ.РФ', 'Global Ports Investments PLC', 'СИНХ-Финанс',
    'ИКС 5 ФИНАНС', 'Страховой Дом ВСК', 'Балтийский лизинг', 'ABH Financial Limited', 
    'Мираторг Финанс', 'Московский кредитный банк', 'Hacienda Investments', 
    'МегаФон Финанс', 'ING Wholesale Banking Russia', 'АВТОБАН-Финанс',
    'Renaissance Financial Holdings Limited', 'Объединенные кондитеры-Финанс'
]

ratings_map = {
    'AAA+': 1, 'AAA-': 1, 'AAA': 1, 
    'AA+': 2, 'AA-': 2, 'AA': 2, 
    'A+': 3, 'A-': 3, 'A': 3, 
    'BBB+': 4,'BBB-': 4, 'BBB': 4, 
    'BB+': 5, 'BB-': 5, 'BB': 5, 
    'B+': 6, 'B-': 6, 'B': 6,
    'CCC': 7, 'CC': 7, 'C': 7,  
    'Withdrawn': 8, 'NR': 8
}

In [ ]:
path = pathlib.Path("/Users/ruaksvb/files/education/data/ratings_hist")

data = []
for iter_path in path.glob("*.pickle"):
    with open(iter_path, 'rb') as f:
        data.extend(pickle.load(f))

rep_date = [x[0][:10] for x in data]
issuer = [x[0].split(",")[0][10:] for x in data]
series = [x[0].split(",")[-1] for x in data]
currency = [x[0].split(",")[-1].strip()[:3] for x in data]
foreign_currencies = ['EUR', 'USD', 'BGN', 'CHF', 'CNY', 'GBP']
is_foreign = [any([c in x[0].split(",")[-1] for c in foreign_currencies]) for x in data]
bond_type = [x[2] for x in data]
rating_type = [x[3].replace("Шкала", "") for x in data]
rating = [x[4].replace("Рейтинг", "").replace(" (Стабильный)", "").replace(" (Негативный)", "") for x in data]
prev_rating = [x[5].replace("Предыдущий рейтинг", "").replace(" (Стабильный)", "").replace(" (Негативный)", "") for x in data]

df_ratings = pd.DataFrame.from_dict(
    {"rating_date": rep_date, 
     "issuer": issuer, 
     "series": series,
     "currency": currency,
     "is_foreign": is_foreign, 
     "bond_type": bond_type, 
     "rating_type": rating_type,
     "rating": rating,
     "prev_rating": prev_rating}
)

df_ratings.drop_duplicates(inplace=True)
df_ratings["year"] = df_ratings["rating_date"].apply(lambda x: x.split(".")[-1])
df_ratings["ticker"] = df_ratings["issuer"].apply(lambda x: tickers.get(x))
df_ratings["num_rating"] = df_ratings["rating"].apply(lambda x: ratings_map.get(x))
assert len(set(list(tickers.values())) - set(df_ratings["ticker"].unique().tolist())) == 0

In [ ]:
df_ratings.to_excel("ratings.xlsx")

### Финансовые показатели компаний

In [ ]:
def parse_report(sec_id, period):
    df_fin = pd.read_html(f"https://smart-lab.ru/q/{sec_id}/f/{period}/MSFO/")[0]
    df_fin = pd.concat([df_fin.loc[:,0], df_fin.loc[:,3:7]], axis=1)
    cols = df_fin.loc[2,:].dropna().values.tolist()
    df_fin = df_fin.loc[3:]
    df_fin.columns=["Label"] + cols
    df_fin.dropna(subset=["Label"], inplace=True)
    df_fin.set_index("Label", inplace=True)
    return df_fin
            

equity_list = [x for x in df_ratings["ticker"].unique().tolist() if x!=None]

fin_reports = dict()
for sec_id in equity_list:
    try:
        df_year = parse_report(sec_id, "y")
        # df_quarter = parse_report(sec_id, "q")
            
        fin_reports[sec_id] = df_year # pd.merge(df_year, df_quarter, left_index=True)
    except Exception as e:
        print(sec_id, e)
        
save_results = True
if save_results:
    df_companies = []
    for sec_id in fin_reports:
        df = fin_reports[sec_id].reset_index()
        df["Company"] = sec_id
        df_companies.append(df)
    df_companies = pd.concat(df_companies)
    df_companies.to_excel("fin_reports_for_ratings.xlsx")

RTSD HTTP Error 404: Not Found
EVRE HTTP Error 404: Not Found


In [ ]:
fin_reports["LNTA"]

,2016,2017,2018,2019,2020
Label,,,,,
Дата отчета,29.06.2018,29.06.2018,22.02.2019,25.02.2020,24.02.2021
Валюта отчета,RUB,RUB,RUB,RUB,RUB
Финансовый отчет,NaN,NaN,NaN,NaN,NaN
Годовой отчет,NaN,NaN,NaN,NaN,NaN
Презентация,NaN,NaN,NaN,NaN,NaN
"Число магазинов, шт",240,328,379,380,393
"Открыто новых магазинов, шт",68,89,51,1,13
"Общ. площадь магазинов, тыс. м2",1 146,1 382,1 467,1 489,1 519
"Сопоставимые продажи, LFL, %",NaN,NaN,NaN,NaN,5.4%


### Подготовка данных

In [ ]:
def extract_field(df, raw, col):
    if raw in df.index:
        output = str(df.loc[raw, col]).replace(" ", "").replace(",", "").replace("%", "")
        if output == "nan":
            output = ""
    else:
        output = ""
    try:
        output = float(output)
    except:
        output = 0
    return output


In [ ]:
def collect_data(year):
    matched_ratings = {}
    for sec_id in equity_list:
        if sec_id in fin_reports.keys() and sec_id not in ["RUAL", "FLOT"]:
            if year in fin_reports[sec_id].columns:
                if isinstance(fin_reports[sec_id].loc["Дата отчета", year], str):
                    report_date = datetime.datetime.strptime(fin_reports[sec_id].loc["Дата отчета", year], "%d.%m.%Y").date()
                    df_iter_ratings = df_ratings.loc[(df_ratings["ticker"]==sec_id)&(~df_ratings["rating"].isin(["Withdrawn", "NR"]))].copy()
                    df_iter_ratings["rating_date"] = pd.to_datetime(df_iter_ratings["rating_date"])                       
                    df_iter_ratings["report_date"] = pd.to_datetime([fin_reports[sec_id].loc["Дата отчета", year]]*df_iter_ratings.shape[0])
                    df_iter_ratings["delta_date"] = (df_iter_ratings["rating_date"] - df_iter_ratings["report_date"]).apply(lambda x: abs(x))
                    min_delta = df_iter_ratings["delta_date"].min()
                    iter_matched_ratins = df_iter_ratings.loc[df_iter_ratings["delta_date"]==min_delta, "rating"].values
                    num_rating = df_iter_ratings.loc[df_iter_ratings["delta_date"]==min_delta, "num_rating"].max()
                    matched_ratings[sec_id] = num_rating
                    #print(sec_id, report_date, min_delta, iter_matched_ratins, num_rating)

    data = {"Рейтинг": matched_ratings}
    
    for col in ["EPS, руб", "Выручка, млрд руб", "CAPEX, млрд руб"]:
        rows = {}
        for sec_id in matched_ratings.keys():
            rows[sec_id] = math.log(max(extract_field(fin_reports[sec_id], col, year), 1))
        data[col] = rows
    
    for col in ["Долг/EBITDA", "ROE, %", "EV/EBITDA", "P/BV", "Див доход, ао, %"]:
        rows = {}
        for sec_id in matched_ratings.keys():
            rows[sec_id] = extract_field(fin_reports[sec_id], col, year)
        data[col] = rows

    df_agg = pd.DataFrame.from_dict(data)
    return df_agg.dropna()

df_data = pd.concat([collect_data("2016"), collect_data("2017"), collect_data("2018"), collect_data("2019"), collect_data("2020")])

#df_agg.groupby("rating")["r_clean"].median().plot()
#df_agg.plot.scatter(x="r_clean", y="rating")

In [ ]:
df_data["key_id"] = df_train.index+df_train["Рейтинг"].apply(lambda x: str(int(x)))
df_val["key_id"] = df_val.index+df_val["Рейтинг"].apply(lambda x: str(int(x)))

In [ ]:
features = ["EPS, руб", "Выручка, млрд руб", "CAPEX, млрд руб",
            "Долг/EBITDA", "ROE, %", "EV/EBITDA", "P/BV", "Див доход, ао, %"]
target = "Рейтинг"

splitter = GroupShuffleSplit(n_splits=1, test_size=0.33)
for train_idx, val_idx in splitter.split(X=df_data[features], y=df_data[target], groups=df_data.index):
    x_train = df_data[features].iloc[train_idx]
    x_val = df_data[features].iloc[val_idx]
    y_train = df_data[target].iloc[train_idx]
    y_val = df_data[target].iloc[val_idx]

### Обучение модели

In [ ]:
#model = RandomForestClassifier(n_estimators=100, random_state=242)
model = LogisticRegression(C=1.0, multi_class="ovr", max_iter=300)
model.fit(X=x_train, y=y_train)

LogisticRegression(max_iter=300, multi_class='ovr')

In [ ]:
warnings.simplefilter("ignore")
print(classification_report(model.predict(x_train), y_train))
print(classification_report(model.predict(x_val), y_val))
warnings.simplefilter("default")

              precision    recall  f1-score   support

         4.0       0.76      0.67      0.71        42
         5.0       0.67      0.72      0.69        47
         6.0       0.79      0.79      0.79        19
         7.0       0.80      1.00      0.89         4

    accuracy                           0.72       112
   macro avg       0.75      0.79      0.77       112
weighted avg       0.73      0.72      0.72       112

              precision    recall  f1-score   support

         4.0       0.82      0.86      0.84        36
         5.0       0.71      0.63      0.67        19
         6.0       1.00      1.00      1.00         5

    accuracy                           0.80        60
   macro avg       0.84      0.83      0.83        60
weighted avg       0.80      0.80      0.80        60



In [ ]:
pd.DataFrame.from_dict({label: {"importance": score} for label, score in zip(features, model.feature_importances_)}, orient="index").sort_values(by="importance", ascending=False)


,importance
"CAPEX, млрд руб",0.194464
"Див доход, ао, %",0.156656
"Выручка, млрд руб",0.153527
P/BV,0.131072
EV/EBITDA,0.097992
Долг/EBITDA,0.097244
"ROE, %",0.085581
"EPS, руб",0.083465


In [ ]:
pd.DataFrame.from_dict({label: {k: v for k, v in zip(features, scores.tolist())} for label, scores in zip(model.classes_, model.coef_)}, orient="index")


,"EPS, руб","Выручка, млрд руб","CAPEX, млрд руб",Долг/EBITDA,"ROE, %",EV/EBITDA,P/BV,"Див доход, ао, %"
4.0,-0.300390,0.118230,1.156174,-1.067192,-0.009916,0.334426,-0.028245,0.331538
5.0,0.157582,0.133306,0.301499,-0.005566,0.001709,0.143737,-0.004495,-0.107158
6.0,0.232958,-0.884042,-1.630468,-0.034367,-0.012289,-0.645595,0.087262,-0.287021
7.0,-0.781750,-0.596933,-1.302192,0.438924,0.002307,0.049560,-0.158051,-0.646964
